In [2]:
import tensorflow as tf

img_size = (224, 224)  
batch_size = 32
#Splitting the dataset into training and validation sets so we can see how well the models are trained
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../../train_val_dataset",
    image_size=img_size,
    batch_size=batch_size,
    label_mode='binary',  
    validation_split=0.2,
    subset="training",
    seed=42
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../../train_val_dataset",
    image_size=img_size,
    batch_size=batch_size,
    label_mode='binary',
    validation_split=0.2,
    subset="validation",
    seed=42
)


Found 2706 files belonging to 2 classes.
Using 2165 files for training.
Found 2706 files belonging to 2 classes.
Using 541 files for validation.


In [3]:
#Assign a certain amount of constant for tuning depending on how much the CPU/GPU can handle so we dont overload it, which could make it slow
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
#Use the sequential model on the dataset where we rescale the images and process it through convolutional layers and max pooling to reduce the dimensions of the picture
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') 
])

c:\Users\Bao Anh\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
#Compile and train the model

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)

Epoch 1/2
68/68 ━━━━━━━━━━━━━━━━━━━━ 59s 829ms/step - accuracy: 0.9308 - loss: 0.3813 - val_accuracy: 0.9519 - val_loss: 0.1187
Epoch 2/2
68/68 ━━━━━━━━━━━━━━━━━━━━ 48s 702ms/step - accuracy: 0.9648 - loss: 0.0975 - val_accuracy: 0.9797 - val_loss: 0.0670
